# load book from pdf file

In [41]:
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt

from whoosh import index
from whoosh.fields import Schema, TEXT
from whoosh.qparser import QueryParser
from transformers import pipeline


schema = Schema(content=TEXT(stored=True))
index_dir = "indexdir"
ix = index.create_in(index_dir, schema)


writer = ix.writer()
# writer.add_document(content="This is the first document.")
# writer.add_document(content="This is the second document.")
# writer.commit()

# Wyszukiwanie dokumentów
# with ix.searcher() as searcher:
#     query = QueryParser("content", ix.schema).parse("first")
#     results = searcher.search(query)
#     for hit in results:
#         print(hit['content'])

# # Użycie modelu językowego do przetwarzania wyników
# nlp = pipeline("feature-extraction")
# for hit in results:
#     embedding = nlp(hit['content'])
#     print(embedding)


#converts pdf, returns its text content as a string
def convert(writer,fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
        page_text = output.getvalue()
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 

# #converts all pdfs in directory pdfDir, saves all resulting txt files to txtdir
# def convertMultiple(pdfDir, txtDir):
#     if pdfDir == "": pdfDir = os.getcwd() + "\\" #if no pdfDir passed in 
#     for pdf in os.listdir(pdfDir): #iterate through pdfs in pdf directory
#         fileExtension = pdf.split(".")[-1]
#         if fileExtension == "pdf":
#             pdfFilename = pdfDir + pdf 
#             text = convert(pdfFilename) #get string of text content of pdf
#             textFilename = txtDir + pdf + ".txt"
#             textFile = open(textFilename, "w") #make text file
#             textFile.write(text) #write text to text file

# # set paths accordingly:
# pdfDir = "./books/"
# txtDir = "./books/"
# convertMultiple(pdfDir, txtDir)
pdf_file = ".\\books\StartingStrength.pdf"
text = convert(pdf_file)

In [ ]:
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os

from whoosh import index
from whoosh.fields import Schema, TEXT
from whoosh.qparser import QueryParser
from transformers import pipeline

# Tworzenie schematu i indeksu
schema = Schema(content=TEXT(stored=True))
index_dir = "index"

# Sprawdzenie, czy katalog indeksu istnieje, jeśli nie, to go tworzymy
if not os.path.exists(index_dir):
    os.mkdir(index_dir)

# Sprawdzenie, czy indeks już istnieje
if index.exists_in(index_dir):
    ix = index.open_dir(index_dir)
else:
    ix = index.create_in(index_dir, schema)

# Dodawanie dokumentów do indeksu
def add_documents(writer, documents):
    for content in documents:
        writer.add_document(content=content)
    writer.commit()

# Konwersja PDF na tekst i dodawanie do indeksu
def convert_and_index(writer, fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    manager = PDFResourceManager()
    laparams = LAParams()
    output_text = []  # Lista do przechowywania tekstu z każdej strony

    infile = open(fname, 'rb')
    for page_number, page in enumerate(PDFPage.get_pages(infile, pagenums), start=1):
        output = StringIO()
        converter = TextConverter(manager, output, laparams=laparams)
        interpreter = PDFPageInterpreter(manager, converter)
        
        interpreter.process_page(page)
        
        # Uzyskujemy tekst z aktualnej strony
        page_text = output.getvalue()
        output_text.append(page_text)  # Dodajemy tekst strony do listy
        # Zamykamy konwerter i bufor po przetworzeniu każdej strony
        converter.close()
        output.close()

    infile.close()

    # Dodajemy przetworzone strony do indeksu
    add_documents(writer, output_text)
    return output_text

# Uruchamianie konwersji i indeksowania
writer = ix.writer()
text_pages = convert_and_index(writer, "./books/StartingStrength.pdf", pages=[1, 2, 3])

# Wyszukiwanie dokumentów
with ix.searcher() as searcher:
    query = QueryParser("content", ix.schema).parse("Squat")
    results = searcher.search(query)
    for hit in results:
        print(hit['content'])

# Użycie modelu językowego do przetwarzania wyników
# nlp = pipeline("feature-extraction")
# for hit in results:
#     embedding = nlp(hit['content'])
#     print(embedding)


In [12]:
with ix.searcher() as searcher:
    query = QueryParser("content", ix.schema).parse("Squat")
    results = searcher.search(query)
    for hit in results:
        print(hit['content'])

Contents 

Strength: Why and How .............................................................................................................. 2 

The Squat ................................................................................................................................ 8 

The Bench Press ......................................................................................................................... 66 

The Deadlift ............................................................................................................................. 104 

The Press .................................................................................................................................. 148 

The Power Clean ......................................................................................................... 168 

Useful Assistance Exercises ....................................................................................................... 208 

Programm

In [1]:
# n = len(text)
# print(n)

# chunk_of_text = text[1000:10000]
# print(chunk_of_text)

In [ ]:
# import sparknlp

# # Start Spark Session
# spark = sparknlp.start()



In [47]:
from transformers import pipeline

# download model
qa_model = pipeline("question-answering")

# test model
question = "How old is Training for strength?"
context = chunk_of_text
qa_response = qa_model(question = question, context = context)
print(qa_response)

C:\Users\Damian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.



{'score': 0.8965405225753784, 'start': 1513, 'end': 1520, 'answer': 'insects'}


In [49]:
file_path = "./books/output.txt"

# Zapis tekstu do pliku
with open(file_path, "w", encoding="utf-8") as file:
    file.write(text)

print(f"Tekst został zapisany do pliku {file_path}")

Tekst został zapisany do pliku ./books/output.txt
